# Kubernetes Networking

In the last notebook, you saw how to manage your pods using Workloads to, for example, control the lifecycle of your pods. However, they were isolated from each other, and anything they were processing were solely based on the arguments we passed to them. 

In order to communicate them, not only amongst themselves, but also from outside the cluster, Kubernetes has Networking resources to expose the pods. 

<p align=center><img src=images/K8_networking.webp width=600></p>

One thing we didn't mention in the last notebook is that each pod has an IP address. Let's check it out

## Try it out

1. If you haven't, start minikube
1. In this [link](https://aicore-files.s3.amazonaws.com/Cloud-DevOps/simple_deployment.yaml), you will find a file named `simple_deployment.yaml`. 
2. Run the right kubectl command to run the deployment
3. Observe the description of the pods using `kubectl get <name of the resource> <name of the pod>`
4. Observe a longer description of the pods using `kubectl get <name of the resource> <name of the pod> -o wide` which stands for output


In [6]:
!kubectl

deployment.apps/hello created


In [18]:
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE
hello-868bcb8b84-575pt   1/1     Running   0          2m39s
hello-868bcb8b84-dp9gr   1/1     Running   0          2m38s
hello-868bcb8b84-vrgqj   1/1     Running   0          2m38s


In [19]:
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE     IP            NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-575pt   1/1     Running   0          2m47s   172.17.0.10   minikube   <none>           <none>
hello-868bcb8b84-dp9gr   1/1     Running   0          2m46s   172.17.0.8    minikube   <none>           <none>
hello-868bcb8b84-vrgqj   1/1     Running   0          2m46s   172.17.0.9    minikube   <none>           <none>


Observe that each pod has an IP address that is only visible if you print out the wide output. 

However, pod IP addresses in Kubernetes are not durable. Whenever an application is scaled up or down, or encounters an error and needs to reboot, their IP address will disappear and need to be reassigned. This change in IP address occurs without warning. In response to this, Kubernetes utilizes Services.

# Services

One of the Networking resources from Kubernetes is 'Services', which in essence, exposes applications to the outside world or eithin the cluster. While Workloads managed the lifecyle of pods, Services manage the lifecycle of the pod's IP addresses. 

Services serve as an abstraction layer on top of the Pods, assigning a single virtual IP address to a specified group of Pods. Once these Pods are associated with that virtual IP address, any traffic which is addressed to that virtual IP will be routed to the corresponding set of Pods. The set of Pods that are linked to a Service can be changed at any time, but the IP address of the Service will remain static.

<p align=center><img src=images/K8_Services.webp></p>

So, how does it translate to our problem? Basically, the IP address changes when the pod dies and restarts, so it would be virtually impossible to point always to a dynamic IP address. Services will not point to the IP address, but to the pod, and from it, it will infer the IP address corresponding to that pod.

Take a look at what happens when we delete a pod from the deployment resource:

## Try it out

1. Observe the IP addresses from the last deployment you created
2. Delete any of the pods in the deployment
3. Observe again the IP addresses

In [20]:
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE   IP            NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-575pt   1/1     Running   0          86m   172.17.0.10   minikube   <none>           <none>
hello-868bcb8b84-dp9gr   1/1     Running   0          86m   172.17.0.8    minikube   <none>           <none>
hello-868bcb8b84-vrgqj   1/1     Running   0          86m   172.17.0.9    minikube   <none>           <none>


In [21]:
!kubectl

pod "hello-868bcb8b84-575pt" deleted


In [22]:
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-dp9gr   1/1     Running   0          86m   172.17.0.8   minikube   <none>           <none>
hello-868bcb8b84-dz5bj   1/1     Running   0          19s   172.17.0.4   minikube   <none>           <none>
hello-868bcb8b84-vrgqj   1/1     Running   0          86m   172.17.0.9   minikube   <none>           <none>


Observe that one of the pods have changed its name AND the IP address, so if we tried to point to the previous IP address, it would point nowhere. 

Services will help us pointing to the pods in the deployment rather than to each individual IP address

There are mainly three services in Kubernetes:

- `ClusterIP`: This is the default Service. It is a fixed IP inside the cluster and it works as a small Load Balancer amongst the resources in the cluster.
- `NodePort`: It is very similar to the `ClusterIP`, but in this case, it creates a port in each node (hence the name) that will receive the traffic and it redirect that traffic to the corresponding pods
- `LoadBalancer`: Exposes the `Service` externally using a cloud provider's load balancer. It gets the resources from the cloud cluster and it redirects the network to our pods

Take a look at this [link](https://kubernetes.io/docs/concepts/services-networking/service/#publishing-services-service-types) to know more about them. In this notebook, we will show you a practical demo on how they work.

## ClusterIP

We are going to use the `Deployment` we ran before as an example. We will use a `ClusterIP` that will connect to the IP addresses of the pods inside the `Deployment`. Here is an example on how to use a `Service` that connects to the `Deployment`

```
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  ports:
  - port: 8080
    targetPort: 8080
  selector:
    role: hello
```

Observe the `spec`s for this Service:

- `ports` specify how to forward requests to pods matching `selector`. In this case:
    - Forward requests to pod's port `8080`
    - Make `Service` itself available under port `8080`
- `selector` points to the resource that we want to get access to.

For the sake of the example, we are going to run a single pod that will run an ubuntu container inside the cluster. That way, we can connect to that cluster, and from the cluster, we will connect to the service:

```
apiVersion: v1
kind: Pod
metadata:
  name: ubuntu
spec:
  containers:
  - name: ubuntu
    image: ubuntu
    args:
    - sleep
    - infinity
```

The example, very broadly, looks like this:

<p align=center><img src=images/ClusterIP.png width=600></p>

## Try it out

1. Create a Service and a pod resources using the configuration mentioned above. _Tip: you can use the same file to create both separating both resources with three dashes (`---`)_
2.  _You should still have the `Deployment` from the before, otherwise, run it now_
3. Run a command to check all pods and services: `kubectl get <your code here>`
4. Run a command to describe the IP addressed pointed by the `Service`: `kubectl <CMD> <resource> <name_of_the_resource>`
5. Run a command to observe all the IP addresses of all pods
6. Compare those addresses with the endpoints pointed by the `Service`
7. Delete one of the pods corresponding to the `Deployment` resource
8. Compare the new addresses with the new endpoints pointed by the `Service`. Is service still pointing to the correct IP addresses?

In [5]:
#Run both resources.
!kubectl

service/hello created
pod/ubuntu created


In [8]:
# Run a command to check all pods and services
!kubectl

NAME                         READY   STATUS    RESTARTS   AGE
pod/hello-868bcb8b84-dnq4w   1/1     Running   0          3m51s
pod/hello-868bcb8b84-j9p5g   1/1     Running   0          3m51s
pod/hello-868bcb8b84-rd6qj   1/1     Running   0          3m51s
pod/ubuntu                   1/1     Running   0          3m43s

NAME                 TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
service/hello        ClusterIP   10.103.95.80   <none>        8080/TCP   3m44s
service/kubernetes   ClusterIP   10.96.0.1      <none>        443/TCP    6h57m

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello   3/3     3            3           3m51s

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-868bcb8b84   3         3         3       3m51s


Observe that the service hello has been created and it has a single cluster IP. However, it doesn't specify which IP addressess it is pointing to

In [9]:
#Run a command to describe the IP addressed pointed by the service:
!kubectl

Name:              hello
Namespace:         default
Labels:            <none>
Annotations:       <none>
Selector:          role=hello
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.103.95.80
IPs:               10.103.95.80
Port:              <unset>  8080/TCP
TargetPort:        8080/TCP
Endpoints:         172.17.0.5:8080,172.17.0.6:8080,172.17.0.7:8080
Session Affinity:  None
Events:            <none>


In [12]:
# Run a command to observe all the IP addresses of all pods
!kubectl

NAME                     READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-dnq4w   1/1     Running   0          10m   172.17.0.7   minikube   <none>           <none>
hello-868bcb8b84-j9p5g   1/1     Running   0          10m   172.17.0.5   minikube   <none>           <none>
hello-868bcb8b84-rd6qj   1/1     Running   0          10m   172.17.0.6   minikube   <none>           <none>
ubuntu                   1/1     Running   0          10m   172.17.0.8   minikube   <none>           <none>


Compare the IP addresses with the endpoints pointed by `Service`.

In [13]:
# Delete one of the pods corresponding to the `Deployment` resource
!kubectl

pod "hello-868bcb8b84-dnq4w" deleted


In [14]:
# Compare the new addresses with the new endpoints pointed by the `Service`
!kubectl 
!kubectl 

NAME                     READY   STATUS    RESTARTS   AGE   IP           NODE       NOMINATED NODE   READINESS GATES
hello-868bcb8b84-j9p5g   1/1     Running   0          15m   172.17.0.5   minikube   <none>           <none>
hello-868bcb8b84-rd6qj   1/1     Running   0          15m   172.17.0.6   minikube   <none>           <none>
hello-868bcb8b84-z7fsc   1/1     Running   0          62s   172.17.0.9   minikube   <none>           <none>
ubuntu                   1/1     Running   0          15m   172.17.0.8   minikube   <none>           <none>
Name:              hello
Namespace:         default
Labels:            <none>
Annotations:       <none>
Selector:          role=hello
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.103.95.80
IPs:               10.103.95.80
Port:              <unset>  8080/TCP
TargetPort:        8080/TCP
Endpoints:         172.17.0.5:8080,172.17.0.6:8080,172.17.0.9:8080
Session Affinity:  None
Events:      

As you can see, the IP addresses have changed, but service is pointing to the new created IP address.

Thus, as mentioned, `Service` is going to create the necessary routes to get the correct IP addresses, even when the pod has been restarted.

Let's see an example that runs on the cluster (which will help us understand the difference between ClusterIP, NodePort, and LoadBalancer). We created a pod called `ubuntu`. If we run a command from the `ubuntu` container, we can go to any of the IP addresses, but the contact point will be `Service`. Let's see how it works

## Try it out

1. Execute bash inside the `ubuntu` pod: `kubectl exec -it ubuntu -- bash` _Run this in the terminal, since the notebook doesn't allow interacting with the CLI_
2. Update and upgrade `ubuntu`: `apt upadte && apt upgrade`
3. Install curl `apt install curl'
4. curl the IP address corresponding to service. In the example above, it would be `http://10.103.95.80:8080` or `http://hello:8080`
5. Repeat step 4 several times, and observe the `Hostname`. 

The output should look something like this:
<p align=center><img src=images/curl.png width=300></p>

Observe that the same IP returns different hostnames. This is because `Service` acts as a load balancer within the cluster and it looks for the first pod that is available.

Let's take a look at NodePort and how it differs from ClusterIP. Before that, make sure to delete the resources that we deployed so far:

In [16]:
!kubectl 
!kubectl 

deployment.apps "hello" deleted
service "hello" deleted
pod "ubuntu" deleted


## NodePort

NodePort will create a port for each node, and that port is exposed to the outer world, so we can connect to the application using our browser!

When you create a service, if you don't specify the type, it will by default be a ClusterIP service. Creating a NodePort is almost identical, except that we need to specify the type of Service, and we _might_ specify the `nodePort` number:

```
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  type: NodePort
  ports:
  - port: 8080
    targetPort: 8080
    nodePort: 30000
  selector:
    role: hello
```

## Try it out

1. Create a .yaml file with the three resources: `ubuntu` pod, `hello` Deployment, and `hello` NodePort
2. Run the file using the right kubectl command
3. Show all the resources you created
4. Get information about the node you have

In [63]:
!kubectl 

deployment.apps/hello created
service/hello created
pod/ubuntu created


In [64]:
# Show all the services you created
!kubectl

NAME                         READY   STATUS    RESTARTS   AGE
pod/hello-868bcb8b84-4jzjf   1/1     Running   0          49s
pod/hello-868bcb8b84-lpvcr   1/1     Running   0          49s
pod/hello-868bcb8b84-shmqz   1/1     Running   0          49s
pod/ubuntu                   1/1     Running   0          49s

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
service/hello        NodePort    10.96.225.205   <none>        8080:30000/TCP   49s
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP          10h

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello   3/3     3            3           49s

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-868bcb8b84   3         3         3       49s


In [31]:
# Get information about the node you have
!kubectl

NAME                         READY   STATUS    RESTARTS   AGE
pod/hello-868bcb8b84-jfjx5   1/1     Running   0          23m
pod/hello-868bcb8b84-mtxt7   1/1     Running   0          23m
pod/hello-868bcb8b84-svqw9   1/1     Running   0          23m
pod/ubuntu                   1/1     Running   0          23m

NAME                 TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
service/hello        NodePort    10.98.208.104   <none>        8080:30000/TCP   23m
service/kubernetes   ClusterIP   10.96.0.1       <none>        443/TCP          9h

NAME                    READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello   3/3     3            3           23m

NAME                               DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-868bcb8b84   3         3         3       23m


You can see that there is no external IP address. This is due to minikube being a testing enviroment and not a deployment environment. In a production environment, the computer in which we deploy our application would show an external IP for us to connect to it, which would be given by Kubernetes' NodePort. 

However, there is a workaound for getting that IP address in minikube:

In [38]:
!minikube service hello

|-----------|-------|-------------|---------------------------|
| NAMESPACE | NAME  | TARGET PORT |            URL            |
|-----------|-------|-------------|---------------------------|
| default   | hello |        8080 | http://192.168.49.2:30000 |
|-----------|-------|-------------|---------------------------|
🏃  Starting tunnel for service hello.
|-----------|-------|-------------|------------------------|
| NAMESPACE | NAME  | TARGET PORT |          URL           |
|-----------|-------|-------------|------------------------|
| default   | hello |             | http://127.0.0.1:53984 |
|-----------|-------|-------------|------------------------|
🎉  Opening service default/hello in default browser...
❗  Because you are using a Docker driver on darwin, the terminal needs to be open to run it.
^C
✋  Stopping tunnel for service hello.

❌  Exiting due to SVC_TUNNEL_STOP: stopping ssh tunnel: os: process already finished

╭────────────────────────────────────────────────────────────

While this is running, minikube will expose the mentioned URL for you to connect to the service. This is in essence what the production environment would do: export an external IP address to which you will add a port, and it will connect you to the pods in the cluster.

Before moving on to the next `Service`, delete all the resources you have created

In [62]:
!kubectl delete -f node_port.yaml

deployment.apps "hello" deleted
service "hello" deleted
pod "ubuntu" deleted


### LoadBalancer

`LoadBalancer` is a way to (more or less) uniformly distribute requests across `Node`s. These are often provided by cloud providers (one can also specify a different one via [`.spec.loadBalancerClass`](https://kubernetes.io/docs/concepts/services-networking/service/#load-balancer-class).

It might look similar to NodePorts, however, LoadBalancers will be helpful for saving you from thinking about the node you want to connect to. LoadBalancers will create a __general__ IP Address that will connect to __any__ of the available nodes. On the other hand, NodePort will create a different IP address for each node, and then you will have to connect to an individual node.

Additionally, LoadBalancers will set the exposed port for you. You can create it using `LoadBalancer` in `spec.type`

```
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  type: LoadBalancer
  ports:
  - port: 8080
    targetPort: 8080
  selector:
    role: hello

In [40]:
!kubectl apply -f node_port.yaml

deployment.apps/hello created
service/hello created
pod/ubuntu created


In [58]:
!kubectl get svc

NAME         TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
hello        LoadBalancer   10.99.190.249   <pending>     8080:31265/TCP   19m
kubernetes   ClusterIP      10.96.0.1       <none>        443/TCP          10h


For cloud environments, one can specify `loadBalancer` via `metadata.annotations` (we will talk about them in more detail later) within your `Service` configuration.

A larger list can be found [here](https://kubernetes.io/docs/concepts/services-networking/service/#internal-load-balancer):

# Ingress

> `Kubernetes` object that manages external access to the services in a cluster

![](./images/ingress-one-service.png)

A few features:
- exposes `HTTP`/`HTTPS` from outside cluster to `Service`s within cluster
- may provide `load balancing`
- may provide `SSL Termination` (decryption of data sent to cluster via `SSL`)

One has to install `ingress controller` of some sorts, available options are described [here](https://kubernetes.io/docs/concepts/services-networking/ingress-controllers/).

If you want to install it in `minikube` run the following command:

In [56]:
!minikube addons enable ingress

💡  After the addon is enabled, please run "minikube tunnel" and your ingress resources would be available at "127.0.0.1"
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/kube-webhook-certgen:v1.0
    ▪ Using image k8s.gcr.io/ingress-nginx/controller:v1.0.0-beta.3
🔎  Verifying ingress addon...
🌟  The 'ingress' addon is enabled


To check that the ingress controller is working fine, you can run:

In [57]:
!kubectl get pods -n ingress-nginx

NAME                                        READY   STATUS      RESTARTS   AGE
ingress-nginx-admission-create--1-z7l8g     0/1     Completed   0          86s
ingress-nginx-admission-patch--1-hwz4q      0/1     Completed   1          86s
ingress-nginx-controller-69bdbc4d57-p5w9g   1/1     Running     0          86s


Ingress will create an nginx controller as well as a load balancer or a NodePort, depending on the platform you use. Observe the following output

In [69]:
!kubectl -n ingress-nginx get svc

NAME                                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                      AGE
ingress-nginx-controller             NodePort    10.106.163.35    <none>        80:31320/TCP,443:31400/TCP   17m
ingress-nginx-controller-admission   ClusterIP   10.104.117.146   <none>        443/TCP                      18m



## Defining resource

As per usual, `.yaml` config:

In [ ]:
---
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: minimal-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /
spec:
  rules:
  - http:
      paths:
      - path: /testpath
        pathType: Prefix
        backend:
          service:
            name: test
            port:
              number: 80

You will see more on Ingress in the practicals!